In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, RidgeCV, LassoCV, ElasticNetCV
from sklearn.metrics import mean_squared_error, make_scorer
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_log_error

from sklearn.linear_model import LinearRegression
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.pipeline import Pipeline
from sklearn.linear_model import Ridge
from xgboost import XGBRegressor
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_log_error
%matplotlib inline

In [ ]:
test = pd.read_csv('test.csv')
data = pd.read_csv('train.csv')

data = pd.get_dummies(data)
test = pd.get_dummies(test)
y = data['Calories']
data.drop('Calories', axis = 1, inplace = True)
data.drop('id', axis = 1, inplace = True)
test.drop('id', axis = 1, inplace = True)
data


,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Sex_female,Sex_male
0,36,189.0,82.0,26.0,101.0,41.0,False,True
1,64,163.0,60.0,8.0,85.0,39.7,True,False
2,51,161.0,64.0,7.0,84.0,39.8,True,False
3,20,192.0,90.0,25.0,105.0,40.7,False,True
4,38,166.0,61.0,25.0,102.0,40.6,True,False
...,...,...,...,...,...,...,...,...
749995,28,193.0,97.0,30.0,114.0,40.9,False,True
749996,64,165.0,63.0,18.0,92.0,40.5,True,False
749997,60,162.0,67.0,29.0,113.0,40.9,False,True
749998,45,182.0,91.0,17.0,102.0,40.3,False,True


In [ ]:
def feature_engineering(df):
    df['bmi'] = df['Weight'] / ((df['Height'] / 100) ** 2)
    df['exercise_intensity'] = df['Heart_Rate'] / df['Duration']
    df['heart_rate_duration'] = df['Heart_Rate'] * df['Duration']
    df['temp_duration'] = df['Body_Temp'] * df['Duration']
    df['hr_to_temp'] = df['Heart_Rate'] / df['Body_Temp']
    df['hr_to_age'] = df['Heart_Rate'] / df['Age']
    df['age_bmi'] = df['Age'] * df['bmi']
    df['max_heart_rate'] = 220 - df['Age']
    df['heart_rate_intensity'] = df['Heart_Rate'] / df['max_heart_rate']

    skewed_feats = ['Age', 'Weight', 'Body_Temp', 'Height', 'Duration', 'Heart_Rate']
    for feat in skewed_feats:
        df[f'log_{feat}'] = np.log1p(df[feat])
    return df
feature_engineering(data)
feature_engineering(test)


,Age,Height,Weight,Duration,Heart_Rate,Body_Temp,Sex_female,Sex_male,bmi,exercise_intensity,...,hr_to_age,age_bmi,max_heart_rate,heart_rate_intensity,log_Age,log_Weight,log_Body_Temp,log_Height,log_Duration,log_Heart_Rate
0,45,177.0,81.0,7.0,87.0,39.8,False,True,25.854639,12.428571,...,1.933333,1163.458776,175,0.497143,3.828641,4.406719,3.708682,5.181784,2.079442,4.477337
1,26,200.0,97.0,20.0,101.0,40.5,False,True,24.250000,5.050000,...,3.884615,630.500000,194,0.520619,3.295837,4.584967,3.725693,5.303305,3.044522,4.624973
2,29,188.0,85.0,16.0,102.0,40.4,True,False,24.049344,6.375000,...,3.517241,697.430964,191,0.534031,3.401197,4.454347,3.723281,5.241747,2.833213,4.634729
3,39,172.0,73.0,20.0,107.0,40.6,True,False,24.675500,5.350000,...,2.743590,962.344511,181,0.591160,3.688879,4.304065,3.728100,5.153292,3.044522,4.682131
4,30,173.0,67.0,16.0,94.0,40.5,True,False,22.386314,5.875000,...,3.133333,671.589428,190,0.494737,3.433987,4.219508,3.725693,5.159055,2.833213,4.553877
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
249995,56,159.0,62.0,6.0,85.0,39.4,True,False,24.524346,14.166667,...,1.517857,1373.363395,164,0.518293,4.043051,4.143135,3.698830,5.075174,1.945910,4.454347
249996,32,202.0,101.0,3.0,84.0,38.4,False,True,24.752475,28.000000,...,2.625000,792.079208,188,0.446809,3.496508,4.624973,3.673766,5.313206,1.386294,4.442651
249997,31,164.0,64.0,14.0,98.0,40.1,True,False,23.795360,7.000000,...,3.161290,737.656157,189,0.518519,3.465736,4.174387,3.716008,5.105945,2.708050,4.595120
249998,62,158.0,61.0,25.0,106.0,40.7,True,False,24.435187,4.240000,...,1.709677,1514.981573,158,0.670886,4.143135,4.127134,3.730501,5.068904,3.258097,4.672829


In [ ]:
y

,Calories
0,150.0
1,34.0
2,29.0
3,140.0
4,146.0
...,...
749995,230.0
749996,96.0
749997,221.0
749998,109.0


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, y, test_size = 0.3, random_state = 0,shuffle=True)

In [ ]:
from sklearn.metrics import make_scorer, mean_squared_log_error

def rmsle_score(y_true, y_pred):
    return np.sqrt(mean_squared_log_error(y_true, y_pred))


#Кросс валидация

In [ ]:
def training(X, y, test, model, cv):
    predictions = []
    scores = []

    for i, (train_idx, val_idx) in enumerate(cv.split(X, y)):
        X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
        y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

        model.fit(X_train, y_train)
        y_pred = model.predict(X_val)
        score = rmsle_score(y_val, y_pred)
        scores.append(score)

        predictions.append(model.predict(test))
        print(f"Fold {i+1} RMSLE: {score:.5f}")

    predictions = np.mean(predictions, axis=0)
    avg_score = np.mean(scores)

    print(f"Average RMSLE: {avg_score:.5f}")
    return predictions, avg_score, model


In [ ]:
SEED = 208
FOLDS = 5
cv = KFold(n_splits=FOLDS, shuffle=True, random_state=SEED)


regressor = TransformedTargetRegressor(
    regressor=XGBRegressor(n_estimators= 300, max_depth=3, learning_rate=0.05),
    func=np.log1p,
    inverse_func=np.expm1
)

pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('poly', PolynomialFeatures(include_bias=False)),
    ('regressor', regressor)
])

predictions, test_rmsle, model = training(data, y, test, pipeline, cv)


Fold 1 RMSLE: 0.06362
Fold 2 RMSLE: 0.06243
Fold 3 RMSLE: 0.06245
Fold 4 RMSLE: 0.06174
Fold 5 RMSLE: 0.06305
Average RMSLE: 0.06266


In [ ]:
test_rmsle

np.float64(0.06265536779149208)

In [ ]:
len(predictions)

In [ ]:
test_df = pd.read_csv('test.csv')
submission = pd.DataFrame({
    "id" : samlpe["id"],
    "Calories": predictions
})
submission.to_csv("submission_ridge.csv", index=False)

In [ ]:
submission